In [1]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import torchvision
import os
from skimage import io
from PIL import Image
import numpy as np

adding some important constants

In [2]:
learning_rate=1e-3
batch_size=32
in_channels=3
num_classes=2
num_epochs=5

In [3]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu') 

Loading a model from torch to use

In [4]:
model = torchvision.models.googlenet(pretrained=True).to(device=device)
# model

Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [7]:
model.fc = torch.nn.Linear(in_features=1024, out_features=2, bias=True).to(device=device)

In [8]:
x = torch.randn(64,3,32,32).to(device=device)
y = model(x)
y.shape

torch.Size([64, 2])

In [9]:
sample_image = io.imread('../input/dogs-cats-images/dataset/training_set/cats/cat.1.jpg')
# sample_image = Image.open('../input/dogs-cats-images/dataset/training_set/cats/cat.1.jpg')

testing model on an image

In [10]:
pi = Image.fromarray(sample_image)

In [11]:
na = np.array(pi)

In [12]:
x = torch.from_numpy(na)
x = x.permute(2,0,1)
x = x.unsqueeze(0)
x.shape

torch.Size([1, 3, 280, 300])

In [14]:
x = x.to(device=device)
y = model(x)
y.shape

torch.Size([1, 2])

creating a custom dataset for cats and dogs images

In [15]:
def loading_images(imagepath):
    images = []
    labels = []
    
    for label, file in enumerate(os.listdir(imagepath)):
        for image_name in os.listdir(os.path.join(imagepath, file)):
            print(os.path.join(imagepath, file, image_name))
            img = io.imread(os.path.join(imagepath, file, image_name))
            images.append(img)
            labels.append(label)
    print(len(images), len(labels))
            
        

In [16]:
# loading_images('../input/dogs-cats-images/dataset/training_set')

In [17]:
class CatsAndDogsDataset(Dataset):
    def __init__(self, imagepath ,transform=None, train=True):
        self.train = train
        self.images = []
        self.labels = []
        self.transform = transform

        for label, file in enumerate(os.listdir(imagepath)):
            for image_name in os.listdir(os.path.join(imagepath, file)):
#                 print(os.path.join(imagepath, file, image_name))
                img = io.imread(os.path.join(imagepath, file, image_name))
                self.images.append(img)
                self.labels.append(label)
        
        self.n_samples = len(self.images)
        print(f"{self.n_samples} images loaded from {imagepath}.")
    
    def __getitem__(self, index):
        
        image = self.images[index]
        pi = Image.fromarray(image)
        
        if (self.transform):
            image = self.transform(pi)
            
        image = np.array(image)
            
        return image, self.labels[index]
        
    
    def __len__(self):
        return self.n_samples

In [19]:
train_dataset = CatsAndDogsDataset('../input/dogs-cats-images/dataset/training_set', transform=transforms.Resize((200,200)))

8000 images loaded from ../input/dogs-cats-images/dataset/training_set.


In [20]:
x,y = train_dataset[0]
x.shape, y

((200, 200, 3), 0)

In [21]:
test_dataset = CatsAndDogsDataset('../input/dogs-cats-images/dataset/test_set', transform=transforms.Resize((200,200)))

2000 images loaded from ../input/dogs-cats-images/dataset/test_set.


In [22]:
train_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)

In [23]:
loss_criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(),lr=learning_rate)

training loop

In [25]:
for epoch in range(num_epochs):
    for data, target in train_dataloader:
        data = data.to(device=device)
#         data = np.array(data)
#         data = torch.from_numpy(data)
        data = data.permute(0,3,2,1)
        target = target.to(device=device)
        
        score = model(data)
#         print(target.shape, score.shape, data.shape, target.shape)
        
        loss = loss_criterion(score, target)
        
        optimizer.zero_grad()
        
        loss.backward()
        
        optimizer.step()
    
    print(f"Loss for current epoch {epoch} : {loss}")

Loss for current epoch 0 : 0.08448906987905502
Loss for current epoch 1 : 0.09245914965867996
Loss for current epoch 2 : 0.13983450829982758
Loss for current epoch 3 : 0.2543966472148895
Loss for current epoch 4 : 0.009369143284857273


In [41]:
def check_accuracy(model, loader):
    model.eval()
    
    total_correct = 0
    total_predictions = 0
    
    with torch.no_grad():
        for x,y in loader:
            x = x.to(device=device)
#             x = np.array(x)
#             x = torch.from_numpy(x)
            x = x.permute(0,3,2,1)
            
            y = y.to(device=device)
            
            score = model(x)
            
            _, prediction = score.max(1)
            
#             print((y==prediction).sum())
            total_correct += (y==prediction).sum()
            total_predictions +=  prediction.size(0)
            
    model.train()
    
    print(f"Accuracy : {float(total_correct/total_predictions)*100:.2f}")
            
        
        

In [42]:
check_accuracy(model, train_dataloader )
check_accuracy(model, test_dataloader)

Accuracy : 98.16
Accuracy : 98.16
